# Fine Tuning Task of a BERT model

### Updating Dependencies
This cell is needed to upgrade the transformers and accelerate packages. If this is not done, the `Trainer` class will not have the correct version and will produce runtime errors.

In [ ]:
! pip install -U datasets
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.0 MB/s eta 0:00:00


### Using a japanese Dataset
We have split the data as follows:
- Training Data with 1000 sentences
- Trianing Data with 3000 sentences
- Evaluation Data with 2000 sentences

The training data shares the same sentences, which is not a problem, because we train two completely different models. The evaluation set contains totally different sentences

In [ ]:
from datasets import load_dataset, Dataset

chosen_language = 'ja' # Japanese (because kaisoku oni ore wa naru!)
train_set_1000 = load_dataset("polyglot_ner", chosen_language, split="train[:1000]")
train_set_3000 = load_dataset("polyglot_ner", chosen_language, split="train[:3000]")
eval_set = load_dataset("polyglot_ner", chosen_language, split="train[7000:9000]")

Generating train split:   0%|          | 0/1691018 [00:00<?, ? examples/s]

### Data Examination
Here, we take a look at the structure, features and labels of the Japanese dataset.

In [ ]:
train_set_1000

Dataset({
    features: ['id', 'lang', 'words', 'ner'],
    num_rows: 1000
})

In [ ]:
train_set_1000_df = train_set_1000.to_pandas()
train_set_1000_df.head()

id lang                               words                       ner
0  0   ja                     [整数, 集合, a, qn]              [O, O, O, O]
1  1   ja  [八幡, 八幡山, 山, 古墳, はちまん, まく, ふん, 日本]  [O, O, O, O, O, O, O, O]
2  2   ja                         [分布, 生息, 域]                 [O, O, O]
3  3   ja                      [北陸, 北陸本線, 本線]                 [O, O, O]
4  4   ja                      [12, 月, 17, 日]              [O, O, O, O]

In [ ]:
features = train_set_1000.features
features

{'id': Value(dtype='string', id=None),
 'lang': Value(dtype='string', id=None),
 'words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [ ]:
# we can drop the id and lang colums, because we only need the sentences and the data
train_set_1000 = train_set_1000.remove_columns(['id', 'lang'])
train_set_3000 = train_set_3000.remove_columns(['id', 'lang'])
eval_set = eval_set.remove_columns(['id', 'lang'])

train_set_1000.features

{'words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [ ]:
def get_all_ner_classes(all_ner_list: list[list[str]]) -> list[str]:
  """
  This method gets all the NE classes, by iterating over all elements
  and collecting them and reducing them to a set.
  The method returns the list of all NE.
  """
    all_ner_classes = []

    for ner_list in all_ner_list:
        for element in ner_list:
            all_ner_classes.append(element)
    return list(set(all_ner_classes))


In [ ]:
ner_classes1000 = get_all_ner_classes(train_set_1000['ner'])
ner_classes3000 = get_all_ner_classes(train_set_3000['ner'])

print(f"The NE classes in the 1000 sentence datase set are {ner_classes1000}.")
print(f"The NE classes in the 3000 sentence datase set are {ner_classes3000}.")
print(f"and they {'are' if ner_classes3000 == ner_classes1000 else 'are not'} identical")

num_ner_classes = len(ner_classes1000) # this is universal for both datasets since they have the same classes
label_to_id = {label: i for i, label in enumerate(ner_classes1000)}


The NE classes in the 1000 sentence datase set are ['ORG', 'PER', 'LOC', 'O'].
The NE classes in the 3000 sentence datase set are ['ORG', 'PER', 'LOC', 'O'].
and they are identical


In [ ]:
import torch
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(ner_classes1000)

max_length = 128

def encode_input(sentences: list[str]) -> torch.Tensor:
    """
    This method concatenates all sentences together and applies the
    Model's tokenizer to get all the needed endoded information, like
    input_ids, attention_mask and input_type_ids
    """
    data = ['. '.join(sentence) for sentence in sentences]
    encoded_inputs = tokenizer(
    data,
    padding='max_length',
    truncation=True,
    max_length=max_length,
    return_tensors='pt'  # Return PyTorch tensors
    )
    return encoded_inputs

def encode_labels(label_list: list[list[str]]) -> torch.Tensor:
    """
    This method returns the encoded labels as a tensor, by using
    sklearn's LabelEncoder Class.
    """
    label_ids = []
    for labels in label_list:
        encoded_labels = list(le.transform(labels))
        while len(encoded_labels) < max_length:
            encoded_labels.append(0)
        label_ids.append(encoded_labels)
    label_tensors = torch.tensor(label_ids, dtype=torch.long)
    return label_tensors


### Chosing the base model
We have decided to use the model below, because it already has a BERT base for japanese implemented, in order to correctly handle japanese input. It is not finetuned, so that we can customize its behavior for our case.

In [ ]:
from transformers import BertTokenizer

bert_japanaese = "cl-tohoku/bert-base-japanese"
tokenizer = BertTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")

input_encoded1000 = encode_input(train_set_1000['words'])
input_encoded3000 = encode_input(train_set_3000['words'])
eval_input_encoded = encode_input(eval_set['words'])


labels1000 = encode_labels(train_set_1000['ner'])
labels3000 = encode_labels(train_set_3000['ner'])
eval_labels = encode_labels(eval_set['ner'])

input_encoded1000['input_ids'][0] , labels1000[0]

tokenizer_config.json:   0%|          | 0.00/104 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


(tensor([    2,  1065,   276,   143,   433,   302,   143,  2187,   143, 18281,
         28550,     3,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

### Creating the Dataset
Due to the already preprocessed data from the Japanese dataset (sentences are already word tokenized) we have to reconstuct a custom `Dataset` Class from huggingface, in order to use the `Trainer` Class properly. We combine the encoded labels and input tokens together in a custom method.

In [ ]:
from datasets import Dataset

def create_dataset(encoded_input, labels):
  """
  Return a HuggingFace Dataset class by combining the encoded input
  and the encoded labels.
  """
  training_data = {
    'input_ids': encoded_input['input_ids'],
    'attention_mask': encoded_input['attention_mask'],
    'token_type_ids': encoded_input['token_type_ids'],
    'labels': labels
  }
  dataset = Dataset.from_dict(training_data)
  return dataset



dataset1000 = create_dataset(input_encoded1000, labels1000)
dataset3000 = create_dataset(input_encoded3000, labels3000)
eval_dataset = create_dataset(eval_input_encoded, eval_labels)



dataset1000

Dataset({
    features: ['input_ids', 'attention_mask', 'token_type_ids', 'labels'],
    num_rows: 1000
})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path_to_data = "/content/drive/MyDrive/ML4NLP/ex05/saved_models"

Mounted at /content/drive


In [ ]:
from transformers import Trainer, TrainingArguments
from torch.optim import AdamW

batch_size = 16

def get_trainer(model, dataset, training_args):
    """
    Return a `Trainer` Class from the base model and dataset
    """
    return Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset= eval_dataset
)

training_args = TrainingArguments(
    output_dir=path_to_data+'/results',
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    fp16=True, # Make this badboy run fast (GPU goes BRRRRRR)
    logging_dir=path_to_data+'/logs'
)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def evaluate_model_and_print_results(trainer):
  predictions, label_ids, metrics = trainer.predict(eval_dataset)
  # Convert logits to predicted label IDs
  predicted_label_ids = np.argmax(predictions, axis=2)

  # Flatten both predictions and true labels for scoring
  true_labels = label_ids.flatten()
  predicted_labels = predicted_label_ids.flatten()

  # Calculate F1 Micro
  f1_micro = f1_score(true_labels, predicted_labels, average='micro')

  # Calculate F1 Macro
  f1_macro = f1_score(true_labels, predicted_labels, average='macro')

  # Calculate Accuracy
  accuracy = accuracy_score(true_labels, predicted_labels)

  print(f"F1 Micro: {f1_micro}")
  print(f"F1 Macro: {f1_macro}")
  print(f"Accuracy: {accuracy}")


In [ ]:
import gc
from transformers import BertForTokenClassification

# define device to work on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_names = ['ner_model_1000_sents', 'ner_model_3000_sents', 'ner_model_3000_sents_frozen']

def train_evaluate_and_save_models():
  """
  Iterate over the model names and build each model iteratively to preserve memory.
  For each model, the according dataset is used. The model is saved in order keep the checkpoints.
  The model's embeddings are frozen if needed.
  """
  for model_name in model_names:
    model = BertForTokenClassification.from_pretrained(bert_japanaese, num_labels=num_ner_classes)
    model.to(device)
    # freeze the third model's embeddings
    if 'frozen' in model_name:
      for param in model.bert.embeddings.parameters():
        param.requires_grad = False

    # chose the correct dataset
    dataset = dataset1000 if '1000' in model_name else dataset3000 # functional expression baaaaby

    trainer = get_trainer(model, dataset, training_args)
    trainer.train()
    model.save_pretrained(path_to_data + '/' + model_name)
    evaluate_model_and_print_results(trainer)

    # trigger Python's garbage collector manually, because we don't trust the inerpreter ;)
    del model
    del trainer
    gc.collect()

In [ ]:
train_evaluate_and_save_models()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


F1 Micro: 0.99423046875
F1 Macro: 0.4830018226484563
Accuracy: 0.99423046875


Some weights of BertForTokenClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


F1 Micro: 0.99653125
F1 Macro: 0.653991122953336
Accuracy: 0.99653125


Some weights of BertForTokenClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


F1 Micro: 0.99671875
F1 Macro: 0.659143548085826
Accuracy: 0.99671875
